In [1]:
import os
os.chdir("..")
print("Current Directory:", os.getcwd())

Current Directory: /workspace/iscat


In [2]:
import h5py
import numpy as np
particle_data_path ='dataset/brightfield_particles.hdf5'
with h5py.File(particle_data_path , 'r') as f:
    print(f['data'].shape)
    print(np.unique(f['labels'],return_counts=True))

(27282, 16, 201)
(array([0, 1, 2]), array([22076,  5146,    60]))


In [3]:
weights = 1/np.array([22076,  5146])

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import h5py
import numpy as np
# from torchvision.models import vit_b_16
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from torchvision.models.vision_transformer import VisionTransformer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, f1_score, balanced_accuracy_score
import matplotlib.pyplot as plt

def compute_normalization_stats(h5_path, classes=None):
    """
    Compute mean and standard deviation for z-score normalization.
    
    Args:
        h5_path (str): Path to HDF5 file
        classes (list, optional): List of classes to include in computation
        
    Returns:
        tuple: (mean, std) computed across all data points
    """
    with h5py.File(h5_path, 'r') as h5_file:
        data = h5_file['data'][:]
        labels = h5_file['labels'][:]
        
        if classes is not None:
            # Filter data for selected classes
            mask = np.isin(labels, classes)
            data = data[mask]
        
        # Compute statistics across all dimensions
        mean = np.mean(data)
        std = np.std(data)
        
        print(f"Computed statistics: mean = {mean:.4f}, std = {std:.4f}")
        
        return mean, std
        
class ParticleDataset(Dataset):
    """Custom Dataset for particle data with flexible class selection and normalization."""
    def __init__(self, h5_path, classes=[0, 1], transform=None, mean=None, std=None):
        self.h5_file = h5py.File(h5_path, 'r')
        data = self.h5_file['data'][:]
        labels = self.h5_file['labels'][:]
        
        # Filter data for selected classes
        mask = np.isin(labels, classes)
        self.data = data[mask]
        self.labels = labels[mask]
        
        # Create class mapping to handle non-consecutive class indices
        self.class_to_idx = {c: i for i, c in enumerate(classes)}
        self.num_classes = len(classes)
        
        # Map original labels to new consecutive indices
        self.labels = np.array([self.class_to_idx[label] for label in self.labels])
        
        self.transform = transform
        self.mean = mean
        self.std = std
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Get particle data
        particle = self.data[idx]  # Shape: (16, 201)
        
        # Apply normalization if mean and std are provided
        if self.mean is not None and self.std is not None:
            particle = (particle - self.mean) / self.std
        
        # First resize to (16, 192) using cubic interpolation
        # Convert to torch tensor for better interpolation
        particle_tensor = torch.FloatTensor(particle).unsqueeze(0)  # Add channel dim
        resized = torch.nn.functional.interpolate(
            particle_tensor.unsqueeze(0),  # Add batch dim
            size=(16, 192),
            mode='bicubic',
            align_corners=True
        ).squeeze(0).squeeze(0)  # Remove batch and channel dims
        
        # Create square tensor with symmetric padding
        target_size = 192
        current_height = resized.shape[0]  # 16
        padding_height = target_size - current_height
        padding_top = padding_height // 2
        padding_bottom = padding_height - padding_top
        
        # Use torch's pad function for symmetric padding
        padded = torch.nn.functional.pad(
            resized,
            (0, 0,                # No padding in width dimension
             padding_top, padding_bottom),  # Padding in height dimension
            mode='constant',
            value=0
        )
        
        # Add single channel dimension
        final_tensor = padded.unsqueeze(0)
        
        if self.transform:
            final_tensor = self.transform(final_tensor)
        
        # Create one-hot encoded label
        label_idx = self.labels[idx]
        label_onehot = torch.zeros(self.num_classes)
        label_onehot[label_idx] = 1
        
        return final_tensor, label_onehot

    def close(self):
        self.h5_file.close()
        
class ModifiedViT(nn.Module):
    def __init__(self, num_classes=2,patch_size=16,num_layers=12,num_heads=12,hidden_dim =768,mlp_dim=3072):
        super().__init__()
        # Load pretrained ViT
        self.vit = VisionTransformer(image_size=192,
        patch_size=16,
        num_classes = num_classes,
        num_layers=12,
        num_heads=12,
        hidden_dim=768,
        mlp_dim=3072,
    )
        
        self.vit.conv_proj = nn.Conv2d(
                    in_channels=1, out_channels=hidden_dim, kernel_size=patch_size, stride=patch_size
                ) 
    def forward(self, x):
        return self.vit(x)
        
def train_model(model, train_loader, val_loader, device, num_epochs=50, weights=None, patience=20):
    criterion = nn.CrossEntropyLoss(weight=torch.Tensor(weights).to(device)) if weights is not None else nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.05)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    best_val_acc = 0.0
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    val_accuracies = []
    val_f1_scores = []
    val_bal_accs = []
    
    early_stopping_counter = 0
    
    for epoch in range(num_epochs):
        # Training Phase
        model.train()
        total_train_loss = 0
        
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} Training'):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        # Validation Phase
        model.eval()
        val_preds = []
        val_true = []
        total_val_loss = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                val_loss = criterion(outputs, labels)
                total_val_loss += val_loss.item()
                
                predicted = torch.argmax(outputs, 1)
                labels = torch.argmax(labels, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_true.extend(labels.cpu().numpy())
        
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        # Calculate metrics
        val_acc = 100 * (sum([p == t for p, t in zip(val_preds, val_true)]) / len(val_true))
        f1 = f1_score(val_true, val_preds, average='weighted')
        bal_acc = balanced_accuracy_score(val_true, val_preds)
        
        # Confusion matrix metrics for class 0
        tp = sum((np.array(val_preds) == 0) & (np.array(val_true) == 0))
        tn = sum((np.array(val_preds) == 1) & (np.array(val_true) == 1))
        fp = sum((np.array(val_preds) == 0) & (np.array(val_true) == 1))
        fn = sum((np.array(val_preds) == 1) & (np.array(val_true) == 0))
        
        val_accuracies.append(val_acc)
        val_f1_scores.append(f1)
        val_bal_accs.append(bal_acc)
        
        # Print epoch details
        print(f'Epoch {epoch+1}:')
        print(f'  Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}')
        print(f'  Val Accuracy = {val_acc:.2f}%, F1-Score = {f1:.4f}, Balanced Acc = {bal_acc:.4f}')
        print(f'  Class 0 Metrics:')
        print(f'    True Positives: {tp}')
        print(f'    True Negatives: {tn}')
        print(f'    False Positives: {fp}')
        print(f'    False Negatives: {fn}')
        print(f'    Precision: {tp/(tp+fp) if (tp+fp) > 0 else 0:.4f}')
        print(f'    Recall: {tp/(tp+fn) if (tp+fn) > 0 else 0:.4f}')
        
        # Early Stopping and Model Saving
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            early_stopping_counter = 0
            torch.save(model.state_dict(), 'best_particle_vit.pth')
        
        # Check for improvement in validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
        
        # Early stopping
        if early_stopping_counter >= patience:
            print(f'Early stopping triggered after {epoch+1} epochs')
            break
        
        scheduler.step()
    
    return (train_losses, val_losses, val_accuracies, 
            val_f1_scores, val_bal_accs, val_preds, val_true)

def plot_training_results(train_losses, val_accuracies, val_f1_scores, val_bal_accs, val_preds, val_true):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
    
    ax1.plot(train_losses, label='Loss')
    ax1.set_title('Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    
    ax2.plot(val_accuracies, label='Accuracy')
    ax2.set_title('Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    
    ax3.plot(val_f1_scores, label='F1-Score')
    ax3.plot(val_bal_accs, label='Balanced Accuracy')
    ax3.set_title('Validation Metrics')
    ax3.set_xlabel('Epoch')
    ax3.legend()
    
    plt.tight_layout()
    plt.savefig('training_plots.png')
    
    cm = confusion_matrix(val_true, val_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix.png')


In [5]:
# Set random seed for reproducibility
torch.manual_seed(42)
DEVICE = "cuda:7"
# Device configuration
device = torch.device(DEVICE if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
classes = [0, 1]
mean, std = compute_normalization_stats('dataset/brightfield_particles.hdf5', classes=classes)
# Load and split dataset
dataset = ParticleDataset(h5_path='dataset/brightfield_particles.hdf5',
                          classes=classes,
                          mean=mean,
                          std=std
                         )
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)
_,c=np.unique(dataset.labels[train_dataset.indices],return_counts=True)
print(c[1]/(c[0]+c[1]))
_,c=np.unique(dataset.labels[val_dataset.indices],return_counts=True)
print(c[1]/(c[0]+c[1]))
# Create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=32,
    shuffle=True,
    num_workers=6,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=6,
    pin_memory=True
)

# Initialize model
model = ModifiedViT(num_classes=len(classes))
model = model.to(device)

# Train model
train_losses, val_accuracies, val_preds, val_true = train_model(
    model, train_loader, val_loader, device
)


Using device: cuda:7
Computed statistics: mean = 7077.7081, std = 1176.7393
0.1875832300133168
0.1948576675849403


Epoch 1/50 Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [02:40<00:00,  4.23it/s]


Epoch 1:
  Train Loss = 0.4275, Val Loss = 0.3457
  Val Accuracy = 83.49%, F1-Score = 0.7893, Balanced Acc = 0.5892
  Class 0 Metrics:
    True Positives: 4348
    True Negatives: 198
    False Positives: 863
    False Negatives: 36
    Precision: 0.8344
    Recall: 0.9918


Epoch 2/50 Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [02:39<00:00,  4.26it/s]


Epoch 2:
  Train Loss = 0.3168, Val Loss = 0.2742
  Val Accuracy = 85.75%, F1-Score = 0.8667, Balanced Acc = 0.8618
  Class 0 Metrics:
    True Positives: 3747
    True Negatives: 922
    False Positives: 139
    False Negatives: 637
    Precision: 0.9642
    Recall: 0.8547


...

Epoch 49/50 Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [02:40<00:00,  4.24it/s]


Epoch 49:
  Train Loss = 0.0244, Val Loss = 0.0808
  Val Accuracy = 97.89%, F1-Score = 0.9789, Balanced Acc = 0.9651
  Class 0 Metrics:
    True Positives: 4330
    True Negatives: 1000
    False Positives: 61
    False Negatives: 54
    Precision: 0.9861
    Recall: 0.9877


Epoch 50/50 Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [02:40<00:00,  4.24it/s]


Epoch 50:
  Train Loss = 0.0241, Val Loss = 0.0809
  Val Accuracy = 97.89%, F1-Score = 0.9789, Balanced Acc = 0.9651
  Class 0 Metrics:
    True Positives: 4330
    True Negatives: 1000
    False Positives: 61
    False Negatives: 54
    Precision: 0.9861
    Recall: 0.9877


ValueError: too many values to unpack (expected 4)

In [6]:
train_losses, val_accuracies, val_preds, val_true

NameError: name 'train_losses' is not defined

In [ ]:
# Plot results
plot_training_results(train_losses, val_accuracies, val_preds, val_true)

# Close HDF5 file
dataset.close()

In [ ]:
dataset = ParticleDataset('dataset/brightfield_particles.hdf5')

In [ ]:
from torchvision.transforms import Resize
import torch
import matplotlib.pyplot as plt
with h5py.File('dataset/brightfield_particles.hdf5', 'r') as f:
    particle_1 = f['data'][f['labels'][:]==0,:][4]
    particle_2 = f['data'][f['labels'][:]==1,:][20]
img = torch.Tensor(particle_1)
print(img.shape)
img = Resize((16,192))(img.unsqueeze(0))
img = img.squeeze(0)
fig, ax = plt.subplots(1, 1, figsize=(30, 5), dpi=300) 
plt.axis('off')
plt.imshow(img,cmap='gray')
plt.show()

In [ ]:
img = torch.Tensor(particle_2)
img = Resize((16,192))(img.unsqueeze(0))
img = img.squeeze(0)
fig, ax = plt.subplots(1, 1, figsize=(30, 5), dpi=300) 
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.show()